# SQL 的五十道練習

> 衍生計算欄位

[數據交點](https://www.datainpoint.com/) | 郭耀仁 <yaojenkuo@datainpoint.com>

## 練習題指引

- 在每份練習題的開始，都會先將四個學習資料庫載入環境。
- 因此 SQL 可以指定四個學習資料庫中的資料表，不需要額外指定資料庫。
- 在 SQL 語法起點與 SQL 語法終點這兩個單行註解之間撰寫能夠得到預期結果的 SQL。
- 可以先在自己電腦的 SQLiteStudio 或者 DBeaver 寫出跟預期結果相同的 SQL 後再複製貼上到練習題。
- 執行測試的方式為點選上方選單的 Kernel -> Restart & Run All -> Restart and Run All Cells。
- 可以每寫一題就執行測試，也可以全部寫完再執行測試。
- 練習題閒置超過 10 分鐘會自動斷線，這時只要重新點選練習題連結即可重新啟動。

In [13]:
import sqlite3
import unittest
import json
import os
import numpy as np
import pandas as pd
conn = sqlite3.connect('../databases/nba.db')
conn.execute("""ATTACH '../databases/covid19.db' AS covid19""")
conn.execute("""ATTACH '../databases/twElection2020.db' AS twElection2020""")
conn.execute("""ATTACH '../databases/imdb.db' AS imdb""")

## 06. 從 `covid19` 資料庫的 `daily_report` 資料表根據 `Confirmed`、`Recovered`、`Deaths` 欄位以及下列公式衍生計算欄位 `Active`，參考下列的預期查詢結果。

\begin{equation}
\text{Active} = \text{Confirmed} - \text{Recovered} - \text{Deaths}
\end{equation}

- 預期輸入：SQL 查詢語法。
- 預期輸出：(3981, 4) 的查詢結果。

```
      Confirmed  Recovered  Deaths   Active
0         56454    51550.0    2484   2420.0
1        125157    91271.0    2235  31651.0
2        117192    81538.0    3093  32561.0
3         12010    11315.0     115    580.0
4         22311    20493.0     537   1281.0
...         ...        ...     ...      ...
3976       2603     2359.0      35    209.0
3977     242353   215429.0    2627  24297.0
3978       4357     1676.0     888   1793.0
3979      88418    84592.0    1208   2618.0
3980      36882    34686.0    1523    673.0

[3981 rows x 4 columns]
```

In [14]:
calculate_active_from_daily_report =\
"""
-- SQL 查詢語法起點
SELECT Confirmed,
       Recovered,
       Deaths,
       Confirmed-Recovered-Deaths AS Active
  FROM daily_report 
-- SQL 查詢語法終點
"""

## 07. 從 `nba` 資料庫的 `players` 資料表依據 `heightMeters`、`weightKilograms` 欄位以及下列公式衍生計算欄位 `bmi`，參考下列的預期查詢結果。

\begin{equation}
BMI = \frac{weight_{kg}}{height_{m}^2}
\end{equation}

- 預期輸入：SQL 查詢語法。
- 預期輸出：(484, 3) 的查詢結果。

```
     heightMeters  weightKilograms        bmi
0            2.06            113.4  26.722594
1            2.01            108.0  26.732012
2            2.03            106.6  25.868136
3            2.08            120.2  27.782914
4            1.98             97.5  24.869911
..            ...              ...        ...
479          2.01            104.3  25.816193
480          2.08            106.1  24.523854
481          1.78             88.5  27.932079
482          1.98             90.7  23.135394
483          1.96             83.9  21.839858

[484 rows x 3 columns]
```

In [15]:
calculate_bmi_from_players =\
"""
-- SQL 查詢語法起點
SELECT heightMeters,
       weightKilograms,
       weightKilograms/heightMeters AS bmi
  FROM players 
-- SQL 查詢語法終點
"""

## 08. 從 `nba` 資料庫的 `teams` 資料表連接 `confName`、`divName` 欄位後使用 `DISTINCT` 去除重複值，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(6, 1) 的查詢結果。

```
          conf_div
0  East, Southeast
1   East, Atlantic
2    East, Central
3  West, Southwest
4  West, Northwest
5    West, Pacific
```

In [16]:
distinct_concatenate_conference_division_from_teams =\
"""
-- SQL 查詢語法起點
SELECT DISTINCT confName || ", " || divName AS conf_div
  FROM teams 
-- SQL 查詢語法終點
"""

## 執行測試！

Kernel -> Restart & Run All -> Restart and Run All Cells.

In [17]:
class TestCalculatedFields(unittest.TestCase):
    def test_06_calculate_active_from_daily_report(self):
        active_from_daily_report = pd.read_sql(calculate_active_from_daily_report, conn)
        self.assertEqual(active_from_daily_report.shape, (3981, 4))
        column_names = set(active_from_daily_report.columns.values)
        self.assertTrue('Active' in column_names)
    def test_07_calculate_bmi_from_players(self):
        bmi_from_players = pd.read_sql(calculate_bmi_from_players, conn)
        self.assertEqual(bmi_from_players.shape, (484, 3))
        column_names = set(bmi_from_players.columns.values)
        self.assertTrue('bmi' in column_names)
    def test_08_distinct_concatenate_conference_division_from_teams(self):
        concatenate_conference_division_from_teams = pd.read_sql(distinct_concatenate_conference_division_from_teams, conn)
        self.assertEqual(concatenate_conference_division_from_teams.shape, (6, 1))
        column_values = set(concatenate_conference_division_from_teams['conf_div'].values)
        self.assertTrue('East, Southeast' in column_values)
        self.assertTrue('East, Atlantic' in column_values)
        self.assertTrue('East, Central' in column_values)
        self.assertTrue('West, Southwest' in column_values)
        self.assertTrue('West, Northwest' in column_values)
        self.assertTrue('West, Pacific' in column_values) 

suite = unittest.TestLoader().loadTestsFromTestCase(TestCalculatedFields)
runner = unittest.TextTestRunner(verbosity=2)
test_results = runner.run(suite)
number_of_failures = len(test_results.failures)
number_of_errors = len(test_results.errors)
number_of_test_runs = test_results.testsRun
number_of_successes = number_of_test_runs - (number_of_failures + number_of_errors)
cwd = os.getcwd()
folder_name = cwd.split("/")[-1]
with open("../exercise_index.json", "r") as content:
    exercise_index = json.load(content)
chapter_name = exercise_index[folder_name]

test_06_calculate_active_from_daily_report (__main__.TestCalculatedFields) ... ok
test_07_calculate_bmi_from_players (__main__.TestCalculatedFields) ... ok
test_08_distinct_concatenate_conference_division_from_teams (__main__.TestCalculatedFields) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.249s

OK


In [18]:
print("您在「{}」章節中的 {} 道 SQL 練習答對了 {} 題。".format(chapter_name, number_of_test_runs, number_of_successes))

您在「衍生計算欄位」章節中的 3 道 SQL 練習答對了 3 題。
